Підвантаження бібліотек

In [ ]:
!pip install PyWavelets matplotlib numpy imageio

import numpy as np
import pywt
import matplotlib.pyplot as plt
import imageio.v3 as iio

np.set_printoptions(suppress=True, precision=3)

Побудуємо початкову матрицю N, та початкову матрицю Хаара (Н) розмірності (4/4):
$$
H = \frac{1}{2}
\begin{bmatrix}
1 & 1 & 1 & 1 \\
1 & 1 & -1 & -1 \\
\sqrt{2} & -\sqrt{2} & 0 & 0 \\
0 & 0 & \sqrt{2} & -\sqrt{2}
\end{bmatrix}
$$



In [ ]:
N = 12
A = np.array([[N, N+1, N+2, N+3],
              [N+4, N+5, N+6, N+7],
              [N+8, N+9, N+10, N+11],
              [N+12, N+13, N+14, N+15]])

H = 1/2 * np.array([[1,  1,  1,  1],
                    [1,  1, -1, -1],
                    [np.sqrt(2), -np.sqrt(2), 0, 0],
                    [0, 0, np.sqrt(2), -np.sqrt(2)]])

Виконаємо двовимірне перетворення Хаара.
Перетворення Хаара (двовимірне) виконується за формулою:
$$
F = H \cdot A \cdot H^T
$$
де:
- (F) — підсумкова матриця після перетворення;
- (A) — початкова матриця;
- (H^T) — транспонована матриця Хаара.


In [ ]:
F = np.dot(np.dot(H, A), H.T)
print("Результат двовимірного перетворення Хаара:\n", F)

Округлити коефіцієнти в отриманій матриці до цілих чисел.

In [ ]:
F_rounded = np.round(F)  # округлення коефіцієнтів матриці F до цілих чисел
print(F_rounded)

Відновити вихідну матрицю з округлених коефіцієнтів та розрахувати
для парних варіантів середньоквадратичне відхилення


In [ ]:
# Відновлення вихідної матриці з округлених коефіцієнтів
A_reconstructed = H.T @ F_rounded @ H
print("Відновлена матриця (округлена):\n", np.round(A_reconstructed))

# Середньоквадратичне відхилення
mse = np.sqrt(np.mean((A - A_reconstructed) ** 2))
print("Середньоквадратичне відхилення (MSE):", mse)

Відновити вихідну матрицю з округлених коефіцієнтів та розрахувати
для непарних – відношення сигнал/шум.

In [ ]:
# Відновлення вихідної матриці з округлених коефіцієнтів
A_reconstructed = H.T @ F_rounded @ H
print("Відновлена матриця (округлена):\n", np.round(A_reconstructed))

# Обчислення сигналу (потужність оригінального сигналу)
signal_power = np.sum(A ** 2)

# Обчислення шуму (відхилення відновленого сигналу)
noise_power = np.sum((A - A_reconstructed) ** 2)

# Відношення сигнал/шум (SNR)
snr = 10 * np.log10(signal_power / noise_power)
print("Відношення сигнал/шум (SNR) в dB:", snr)


Приклад програмного рішення в python, що є аналогом Wavelet_Compression з Matlab

In [ ]:

# Відкриваємо зображення і конвертуємо у відтінки сірого
image = iio.imread('./example.jpg', mode='L')


# Перетворення даних у масив numpy й нормалізація
data = np.array(image, dtype=float) / 255.0


# Виконання вейвлет-перетворення
coeffs = pywt.wavedec2(data, wavelet='haar', level=2)

# Встановлення порога для компресії (трешхолдінг)
threshold = 0.05
coeffs_thresh = list(coeffs)
coeffs_thresh[0] = pywt.threshold(coeffs_thresh[0], threshold, mode='soft')
coeffs_thresh[1:] = [(pywt.threshold(cH, threshold, mode='soft'),
                      pywt.threshold(cV, threshold, mode='soft'),
                      pywt.threshold(cD, threshold, mode='soft')) for cH, cV, cD in coeffs_thresh[1:]]

# Зворотне вейвлет-перетворення для відновлення даних
compressed_data = pywt.waverec2(coeffs_thresh, wavelet='haar')

# Візуалізація вихідного та стисненого зображень
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(data, cmap='gray')
plt.title('Оригінальні дані')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(compressed_data, cmap='gray')
plt.title('Стиснені дані (після thresholding)')
plt.axis('off')

plt.show()
